# MLFlow example

Let's see how we can use MLFlow with Sinergym, first the imports:

In [ ]:
import mlflow

import gym
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import CallbackList
from stable_baselines3.common.vec_env import DummyVecEnv

from datetime import datetime

from sinergym.utils.callbacks import LoggerEvalCallback
from sinergym.utils.rewards import *
from sinergym.utils.wrappers import LoggerWrapper
import sinergym

Then lets define the name/string variables we are going to use

In [ ]:
environment = "Eplus-demo-v1"
episodes = 4
experiment_date = datetime.today().strftime('%Y-%m-%d %H:%M')
name = F"DQN-{environment}-episodes_{episodes}({experiment_date})"

Now we are going to use the same DRL code/example as the 'DRL usage example' but this time with mlflow, for that we need to wrap the code with:
with mlflow.start_run(run_name=name) as shown next:

In [ ]:
with mlflow.start_run(run_name=name):
    env = gym.make(environment, reward=LinearReward)
    env = LoggerWrapper(env)

    # Defining model(algorithm)
    model = DQN('MlpPolicy', env)

    # Calculating n_timesteps_episode for training
    n_timesteps_episode = env.simulator._eplus_one_epi_len / \
                          env.simulator._eplus_run_stepsize
    timesteps = episodes * n_timesteps_episode

    # For callbacks processing
    env_vec = DummyVecEnv([lambda: env])

    # Using Callbacks for training
    callbacks = []

    # Set up Evaluation and saving best model
    eval_callback = LoggerEvalCallback(
        env_vec,
        best_model_save_path='best_model/' + name + '/',
        log_path='best_model/' + name + '/',
        eval_freq=n_timesteps_episode * 2,
        deterministic=True,
        render=False,
        n_eval_episodes=2)
    callbacks.append(eval_callback)

    callback = CallbackList(callbacks)

    # Training
    model.learn(
        total_timesteps=timesteps,
        callback=callback,
        log_interval=1)
    model.save(env.simulator._env_working_dir_parent + '/' + name)

    # End mlflow run
    mlflow.end_run()